# Introdução

Projeto final da cadeira IF807 que tem como o objetivo a implementação de uma técnica dentro do tema de Fairness em sistemas de recomendação. A técnica é baseada em um artigo que pode ser encontrado nesse link: https://dl.acm.org/doi/10.1145/3292500.3330691

Equipe:



O objetivo é implementar um sistema de re-ranqueamento que consideram o Fainess, para que então ocorra menos viéses na hora de gerar listas de ranking em algoritmos de recomendação. Com essa proposta, esse viés é quantificado e mitigado, para um determinado subconjunto de dados, esses algoritmos podem gerar uma distribuição desejada dos resultados mais bem classificados, mantendo assim a paridade demográfica ou a igualdade de oportunidades conforme necessário. 

As métricas são definidas para avaliar o nível de Fairness então alcançado e são mostrador resultados obtidos através de dados sinteticamente gerados que buscam replicar uma aplicação prática de uma busca de talentos do LinkedIn.

# Implementação

### Bibliotecas

In [3]:
import numpy as np
import scipy
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

### Medidas para avaliação de viés

O artigo lista algumas medidas que podem ser utilizadas para obter o valor de viés de cada um dos algoritmos de recomendação, eles são definidos como o seguinte:

##### Medidas baseada nos top-k resultados

##### Skew@k

O Desvio dos top-k resultados rankeados da proporção desejada é quantificada usando uma métrica skew (assimetria). A função log retorna um valor negativo de assimetria para under representation ou positiva para over representation do atributo que está sendo avaliado. 

In [4]:
def proportion_calc(df,k,ai):
    count=0
    for i in range(k):  
        if (df[i] == ai): 
            count = count+1
    return (count/k)    

def skew_func(df,p,k,ai):
    s = math.log((proportion_calc(df,k,ai)+1)/(p[ai]+1)) 
    return s

A métrica acima tem alguns problemas, primeiro o fato de que é definida apenas sobre o valor de um atributo e precisa ser estendida para aglobar vários, e também, o valor computado depende altamente do valor k (top valores) e então uma métrica acumulativa e compreensiva é requerida. Para resolver esses problemas, são definidas métricas MinSkew e MaxSkew que quantificam a pior desvantagem e maior vantagem para um candidato.

##### MinSkew@k

In [5]:
def min_skew(df,p,k):
    min_skew_var=float('inf')
    for x in range(len(p)):
        m=skew_func(list(df),p,k,x)
        if m < min_skew_var:
            min_skew_var = m
    return min_skew_var

##### MaxSkew@k

In [6]:
def max_skew(df,p,k):
    max_skew_var=-float('inf')
    for x in range(len(p)):
        m=skew_func(list(df),p,k,x)
        if m > max_skew_var:
            max_skew_var = m
    return max_skew_var

Para o segundo problema citado acima, métricas de ranqueamento são definidas e listadas abaixo.

### Medidas de ranqueamento

##### NDKL

A divergência Kullback-Leibler (KL) é uma medida não negativa no qual o valor maior denota uma grande divergência entre as distribuições, ela é implementada como uma medida acumulativa envolvendo uma média baseada em pesos de Skew@i sobre todos os valores de atributo.

Um viés de magnitude igual mas em direções opostas não pode ser distinguido por essa métrica, por isso ele não indica qual atributo está sendo tratado de forma injusta.

In [7]:
def kld_func(D1,D2):
    a = np.asarray(D1, dtype=np.float)
    b = np.asarray(D2, dtype=np.float)

    return np.sum(np.where(a != 0 , a * np.log((a +0.00001)/(b+0.00001)), 0))

def ndkl_func(df,p):
    Z = np.sum(1/(np.log2(np.arange(1,len(df)+1)+1)))
    total=0

    for i in range(1,len(df)+1): 
        value=df[:i].value_counts(normalize = True)
        value=value.to_dict()
        D1=[]
        for i in range(len(p)):
            if i in value.keys():
                D1.append(value[i])
            else:
                D1.append(0)
        total=total+(1/math.log2(i+1)) * kld_func(D1,p)

    return (1/Z)*total

##### NDCG

Ganho de desconto acumulativo normalizado, o ganho cumulativo captura que os resultados muito relevantes são mais úteis do que os resultados parcialmente relevantes, que, por sua vez, são mais úteis do que os resultados irrelevantes. Dependendo da noção de relevância, que para o caso do teste é o score de cada candidato.

In [8]:
def dcg_at_k(r, k, method=0):
    r = np.asfarray(r)[:k]
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0

def ndcg_at_k(df, k, method=0):
    r=list(df)
    dcg_max = dcg_at_k(sorted(r, reverse=True), k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max

### Propriedades desejadas

Para garantir uma representação justa nos resultados top-k de uma lista classificada, é importante que a proporção de cada atributo protegido (como gênero ou raça) esteja dentro de limites específicos. Se qualquer top-k violar essas proporções mínimas, o sistema é considerado inviável. Duas medidas ajudam a avaliar isso: o Índice Inviável, que conta quantas vezes qualquer top-k quebra a regra de representação mínima, e a Contagem Inviável, que contabiliza todas as violações de representação mínima em cada posição top-k. Essas métricas podem ser ajustadas para diferentes tamanhos de listas e quantidades de atributos para garantir comparações justas.

In [10]:
def infeasible_index(df, p):
    
    # Parâmetros e variáveis iniciais
    data = df[:100]
    num_attributes = len(p)
    tao_r = 100
    inf_index_tao_r = 0
    inf_count_tao_r = 0

    # Iterar sobre cada top-k
    for k in range(1, tao_r + 1):
        data_temp_k = data[:k]
        infeasible_flag = False

        # Verificar cada valor de atributo protegido
        for i, desired_proportion in enumerate(p):
            observed_count_ai = (data_temp_k['ai'] == i).sum()
            desired_count_ai = math.floor(desired_proportion * k)

            # Atualizar flags e contadores se a condição mínima não for atendida
            if observed_count_ai < desired_count_ai:
                infeasible_flag = True
                inf_count_tao_r += 1

        # Incrementar índice inviável se houver uma violação
        if infeasible_flag:
            inf_index_tao_r += 1

    # Mensagem de erro caso o índice inviável seja maior que 99
    if inf_index_tao_r > 99:
        print(f"Erro em {num_attributes} e é: {inf_index_tao_r}")

    return [inf_index_tao_r, inf_count_tao_r]


### Algoritmos de re-Ranqueamento

##### DetGreedy

O algoritmo DetGreedy funciona assim:

Primeiro, se a representação mínima de um atributo protegido não está sendo cumprida, selecione o item com a maior pontuação disponível.
Caso contrário, selecione o item com a maior pontuação disponível entre os que ainda não atingiram o limite máximo de representação.
Atenção: Embora este algoritmo busque maximizar as pontuações dentro de cada grupo de atributos protegidos, ele pode acabar violando as restrições de representação, tornando a lista final inviável.

In [13]:
def det_greedy(data,p,k_max):
    rankedAttList = [] 
    rankedScoreList = []
    counts={}
    for i in range(len(p)):
        counts[i]=0
    for k in range(1,k_max+1):
        belowMin = {ai for ai,v in counts.items() if v < math.floor(k*p[ai]) }
        belowMax = {ai for ai,v in counts.items() if v >= math.floor(k*p[ai]) and v <math.ceil(k*p[ai]) }
        s={}
        if len(belowMin) != 0:
            for i in belowMin:
                s[i]=data[(i,counts[i])]
            nextAtt = max(s,key=s.get)
        else:
            for i in belowMax:
                s[i]=data[(i,counts[i])]
            nextAtt = max(s,key=s.get)
        rankedAttList.append(nextAtt)
        rankedScoreList.append(data[(nextAtt,counts[nextAtt])])
        counts[nextAtt]+=1
        
    return pd.DataFrame(list(zip(rankedAttList, rankedScoreList)),columns =['ai', 'score']) 

##### DetCons

O algoritmo DetCons é utilizado para verificar a consistência de um conjunto de restrições binárias em problemas de satisfação de restrições (CSP). Ele revisa cada restrição binária para garantir que nenhum valor possa ser removido de um domínio de variável sem violar a consistência dessa restrição. Este processo é repetido até que nenhum valor possa ser removido sem quebrar a consistência de alguma restrição, assegurando assim a solução consistente do CSP.

In [14]:
def det_cons(data,p,kmax):

    rankedAttrList=[]
    rankedScoreList=[]
    counts={}
    for i in range(len(p)):
        counts[i]=0
    
    for k in range(1,kmax+1):
        belowMin = {ai for ai,v in counts.items() if v < math.floor(k*p[ai]) }
        belowMax = {ai for ai,v in counts.items() if v >= math.floor(k*p[ai]) and v <math.ceil(k*p[ai]) }
        s={}
        if len(belowMin)!=0:
            for i in belowMin:
                s[i]=data[(i,counts[i])]
            nextAtt= max(s,key=s.get)
                
        else: 
            s={}
            for i in belowMax:
                s[i]=math.ceil(i*p[i])/p[i]
                
            nextAtt= min(s,key=s.get)
                
        rankedAttrList.append(nextAtt)
        rankedScoreList.append(data[(nextAtt,counts[nextAtt])])
        counts[nextAtt]+=1
        
    return pd.DataFrame(list(zip(rankedAttrList, rankedScoreList)),columns =['ai', 'score'])